<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 4.1.2 Linear Regression

## Prediction of House Price Using Linear Regression

### Data

The Sacramento real estate transactions file is a list of 985 real estate transactions in the Sacramento area reported over a five-day period, as reported by the Sacramento Bee.

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

### 1. Explore Data

Perform EDA on this dataset. Find Dependent and Independent Variables to predict house price.

In [2]:
# Read CSV
house_csv = "Sacramento_transactions.csv"

In [3]:
#dataframe
raw = pd.read_csv(house_csv)
raw.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [4]:
# finding how many different unique values each of the object columns have

print(f"Numbers of unique values: \nCity: {raw['city'].nunique()}\nZip: {raw['zip'].nunique()}\nState: {raw['state'].nunique()}\nType: {raw['type'].nunique()}")
      
# city and zip have too many values to cover and I thnk latitude and longitude can subsitute for where the house is placed.
# since state is all in the same area there is no need to use it to predict

Numbers of unique values: 
City: 39
Zip: 68
State: 1
Type: 4


In [5]:
#finding the 4 different types of house
raw['type'].unique()

array(['Residential', 'Condo', 'Multi-Family', 'Unkown'], dtype=object)

In [6]:
# finding how many of the houses have unknown type
raw[raw['type'] == 'Unkown']

#unknown type can most likely be dropped since there is only the price but nothing else to really help predict the price

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
757,6007 MARYBELLE LN,SHINGLE SPRINGS,95682,CA,0,0,0,Unkown,Fri May 16 00:00:00 EDT 2008,275000,38.64347,-120.888183


In [7]:
# drop the unknown type house
df = raw.drop([raw.index[757]])

df[756:]

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
756,2674 TAM O SHANTER DR,EL DORADO HILLS,95762,CA,4,2,0,Residential,Fri May 16 00:00:00 EDT 2008,272700,38.695801,-121.079216
758,9949 NESTLING CIR,ELK GROVE,95757,CA,3,2,1543,Residential,Fri May 16 00:00:00 EDT 2008,275000,38.397455,-121.468391
759,2915 HOLDREGE WAY,SACRAMENTO,95835,CA,5,3,2494,Residential,Fri May 16 00:00:00 EDT 2008,276000,38.663728,-121.525833
760,2678 BRIARTON DR,LINCOLN,95648,CA,3,2,1650,Residential,Fri May 16 00:00:00 EDT 2008,276500,38.844116,-121.274806
761,294 SPARROW DR,GALT,95632,CA,4,3,2214,Residential,Fri May 16 00:00:00 EDT 2008,278000,38.258976,-121.321266
...,...,...,...,...,...,...,...,...,...,...,...,...
980,9169 GARLINGTON CT,SACRAMENTO,95829,CA,4,3,2280,Residential,Thu May 15 00:00:00 EDT 2008,232425,38.457679,-121.359620
981,6932 RUSKUT WAY,SACRAMENTO,95823,CA,3,2,1477,Residential,Thu May 15 00:00:00 EDT 2008,234000,38.499893,-121.458890
982,7933 DAFFODIL WAY,CITRUS HEIGHTS,95610,CA,3,2,1216,Residential,Thu May 15 00:00:00 EDT 2008,235000,38.708824,-121.256803
983,8304 RED FOX WAY,ELK GROVE,95758,CA,4,2,1685,Residential,Thu May 15 00:00:00 EDT 2008,235301,38.417000,-121.397424


In [8]:
# how many values of the data is missing
raw[raw == 0].count()

street         0
city           0
zip            0
state          0
beds         108
baths        108
sq__ft       171
type           0
sale_date      0
price          0
latitude       0
longitude      0
dtype: int64

In [11]:
# drop the missing values
df_remove_na = raw.drop(raw[raw['sq__ft'] == 0].index)
df_remove_na.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [12]:
# how many rows are left
df_remove_na.shape

(814, 12)

In [13]:
# create DF with only predictor variables
house_df = df_remove_na \
    .drop(columns=['street','city','state','sale_date']) \
    .pipe(lambda x: pd.get_dummies(x, drop_first=True))
house_df.head()

,zip,beds,baths,sq__ft,price,latitude,longitude,type_Multi-Family,type_Residential
0,95838,2,1,836,59222,38.631913,-121.434879,0,1
1,95823,3,1,1167,68212,38.478902,-121.431028,0,1
2,95815,2,1,796,68880,38.618305,-121.443839,0,1
3,95815,2,1,852,69307,38.616835,-121.439146,0,1
4,95824,2,1,797,81900,38.519470,-121.435768,0,1


### 2. Predict Price

We are going to predict `price` from available information.

#### 2.1 What is the Target Variable? Why?

In [14]:
# ANSWER price because we want to predict the price from other information avaliable


#### 2.2 List all possible variables which might be Independent/Predictor variable.

In [15]:
# ANSWER all numerical columns except for price, type can be changed into numerical data e.g. 1 = residental etc.
#zip, beds, baths, sq__ft, type, latitude, longitude

#### 2.3 Find correlation between variables

Find correlation between variables. Which is the best predictor? Why? State your reason.

In [16]:
# ANSWER best variable would be the number sq_ft
house_df.corr()['price']

zip                 -0.219591
beds                 0.441988
baths                0.501820
sq__ft               0.693708
price                1.000000
latitude             0.058475
longitude            0.278312
type_Multi-Family   -0.005227
type_Residential     0.166090
Name: price, dtype: float64

#### 2.4 Find Coefficient and Intercept using Linear Regression

Using LinearRegression of sklearn.linear_model package find Coefficient and Intercept.

Create an instance of LinearRegression.

Explore following methods:

- fit
- predict
- score

In [17]:
from sklearn.linear_model import LinearRegression

##### 2.4.1 Fit predictor and target variables using linear regression 

In [18]:
# ANSWER
# Take all columns except target as predictor columns
predictor_columns = [c for c in house_df.columns if c != 'price']
# Load the dataset as a pandas data frame
X = pd.DataFrame(house_df, columns = predictor_columns)

# Target Variable
Y = house_df['price']

model = LinearRegression()

In [19]:
fit = model.fit(X, Y)
fit

LinearRegression()

##### 2.4.2 Find R Score

Find $R^2$ Using ```score``` method of Linear Regression.

In [20]:
# ANSWER
fit.score(X, Y)

0.5338056621846419

### 3. Splitting Data

In [21]:
from sklearn.model_selection import train_test_split

#### 3.1 Create training and testing subsets

Hint: Using `train_test_split` Library.

```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
```

In [22]:
# ANSWER
## Create training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

#### 3.2 Check Shape, Sample of Test Train Data

In [23]:
# ANSWER
## Check training/test data
print(f'X train shape: {X_train.shape}')
print(f'y train shape: {y_train.shape}')
print(f'X test shape: {X_test.shape}')
print(f'y test shape: {y_test.shape}')

X train shape: (545, 8)
y train shape: (545,)
X test shape: (269, 8)
y test shape: (269,)


#### 3.3 Using Linear Regression Find The Score

1. Fit model using X_train, y_train
2. Find score using X_test, y_test

In [24]:
# ANSWER
fit_train = model.fit(X_train, y_train)

In [25]:
# ANSWER
fit_train.score(X_test, y_test)

0.5264691612626062

### 4. Predict House Price

Let's assume we have information of following house:

- street:	1140 EDMONTON DR
- city:	SACRAMENTO
- zip:	95833
- state:	CA
- beds:	3
- baths:	2
- sq__ft:	1204
- type:	Residential

**Predict the price of this house using linear regression model.**

In [26]:
# ANSWER
x = pd.DataFrame(np.array([95833, 3, 2, 1204, 38.624570, -121.486913, 0, 1]).reshape(1, -1), columns =
                 ['zip', 'beds', 'baths', 'sq__ft', 'latitude', 'longitude', 'type_Multi-Family', 'type_Residential'])
y_pred = fit.predict(x)
print(f"predicted response:\n{y_pred}")

predicted response:
[167816.33956509]


#### Find the error

In [29]:
# Predict price from X_test
actual_price = raw[raw['street'] == '1140 EDMONTON DR']['price']

In [30]:
Error = y_pred - actual_price
Error

930   -6433.660435
Name: price, dtype: float64



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



